In [29]:
from dotenv import load_dotenv

import PyPDF2
import sqlite3

import numpy as np
import pandas as pd
import pickle

from openai import OpenAI
import os

import streamlit as st

In [30]:
def data_converter(row):
    system_prompt = """
    you are a Bob Dylan poetry generator bot. 
    Please generate a poem in Bob Dylan's style. The topic is: 
    """
    user_prompt = row['title']
    lyrics = row['lyrics']

    return {
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
            {"role": "assistant", "content": lyrics}
        ]
    }

In [31]:
data_path = '../data/Bob_Dylan.csv'
data = pd.read_csv(data_path)
data = data.dropna(subset=['lyrics'])

data_json = data.apply(data_converter, axis=1)

In [32]:
# save data_json to jsonl file
output_dir = '../data/Bob_Dylan.jsonl'
data_json.to_json(output_dir, orient='records', lines=True)

# Fine tuning

In [33]:
from openai import OpenAI

load_dotenv()

client = OpenAI()

client.files.create(
  file=open(output_dir, "rb"),
  purpose="fine-tune"
)

FileObject(id='file-vaERUmreiSmqaDVaE4yNYFFT', bytes=703787, created_at=1711149750, filename='Bob_Dylan.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [34]:
client.fine_tuning.jobs.create(
  training_file='file-vaERUmreiSmqaDVaE4yNYFFT', 
  model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-vd9M7mp3VHThi4ueOCbukgZV', created_at=1711149762, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-g4KwJifjAStr9RyEir7UqpEP', result_files=[], status='validating_files', trained_tokens=None, training_file='file-vaERUmreiSmqaDVaE4yNYFFT', validation_file=None, user_provided_suffix=None)

In [35]:
client.fine_tuning.jobs.list(limit=10)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-vd9M7mp3VHThi4ueOCbukgZV', created_at=1711149762, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-g4KwJifjAStr9RyEir7UqpEP', result_files=[], status='validating_files', trained_tokens=None, training_file='file-vaERUmreiSmqaDVaE4yNYFFT', validation_file=None, user_provided_suffix=None), FineTuningJob(id='ftjob-AXia4OhhM4YYJnFGc9nKOlCN', created_at=1711148358, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-g4KwJifjAStr9RyEir7UqpEP', result_files=[], status='running', trained_tokens=None,

In [ ]:
completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo:my-org:custom_suffix:id",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)
print(completion.choices[0].message)